In [1]:
import os
import pandas as pd

import torch
import torch.nn as nn
import torch.functional as F

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score, roc_auc_score
from sklearn.metrics import RocCurveDisplay

import matplotlib.pyplot as plt
import seaborn

In [2]:
class Net256(nn.Module):
    def __init__(self, n_classes):
        super(Net256, self).__init__()

        # input layer takes the first 256 principal components 
        self.lin_in = nn.Linear(256, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.relu1 = nn.ReLU()

        # 1. hidden layer
        self.hidden1 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.relu2 = nn.ReLU()

        # 2. hidden layer
        self.hidden2 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.relu3 = nn.ReLU()

        # output layer
        self.lin_out = nn.Linear(64, n_classes)

    def forward(self, x):
        # input layer
        x = self.relu1(self.bn1(self.lin_in(x)))

        # first hidden layer
        x = self.relu2(self.bn2(self.hidden1(x)))

        # second hidden layer
        x = self.relu3(self.bn3(self.hidden2(x)))

        # output layer. We don't add a softmax layer because we weill use
        # cross entropy loss as a loss funtion
        out = self.lin_out(x)

        return out

In [17]:
random_state = 764
n_classes = 12 # number of MoA

learning_rate = 1e-4
n_epochs = 10
batch_size = 32
chunk_print = 200

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [4]:
model = Net256(n_classes)
print(model)

Net256(
  (lin_in): Linear(in_features=256, out_features=256, bias=True)
  (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (hidden1): Linear(in_features=256, out_features=128, bias=True)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (hidden2): Linear(in_features=128, out_features=64, bias=True)
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (lin_out): Linear(in_features=64, out_features=12, bias=True)
)


In [5]:
# using cross entropy loss for multiclass classification
loss_func = nn.CrossEntropyLoss()

# use Adam as optimizer for this NN
optimizer = torch.optim.Adam(model.parameters(), foreach=True, lr=learning_rate)

/home/mhuebsch/.conda/envs/ms2/lib/python3.12/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [16]:
# emb_vec = torch.rand(32, 256)
# result = model(emb_vec)
# print(result)

In [ ]:
print("Begin Training")

model.train()
for epoch in range(n_epochs):
    running_loss = 0.0

    # output = model(input)
    optimizer.zero_grad()
    loss = loss_func(output, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    
    if (i+1) % chunk_print == 0:
      print(f"Epoch [{epoch+1}/{n_epochs}], Step [{i+1}/{len(ds_train)//batch_size}], Loss: {running_loss/chunk_print}")
      running_loss = 0.0


print("Stop Training")